In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow transformers datasets pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/Twitter_Sentiment_Analysis/stemmed_twitter_data.csv')

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(texts):
    # Ensure that texts is a list of strings
    if isinstance(texts, str):
        texts = [texts]  # Convert a single string to a list of one string
    return tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="tf")

# Convert to string and handle NaNs
train_df['stemmed_content'] = train_df['stemmed_content'].fillna('').astype(str)
test_df['stemmed_content'] = test_df['stemmed_content'].fillna('').astype(str)

# Tokenize the datasets
train_encodings = tokenize(train_df['stemmed_content'].tolist())
test_encodings = tokenize(test_df['stemmed_content'].tolist())




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import tensorflow as tf

def convert_to_tf_dataset(encodings, labels):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask']
        },
        labels
    ))
    return dataset

train_dataset = convert_to_tf_dataset(train_encodings, train_df['target'].values)
test_dataset = convert_to_tf_dataset(test_encodings, test_df['target'].values)

# Configure the datasets for performance
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
import tensorflow as tf
from transformers import TFAutoModel

class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes=2):
        super(BERTForClassification, self).__init__()
        self.bert = bert_model
        # Freeze the BERT model to prevent training
        self.bert.trainable = False
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        output = self.bert(inputs)
        pooled_output = output.pooler_output
        pooled_output = self.dropout(pooled_output)
        return self.classifier(pooled_output)

# Load the BERT model
bert_model = TFAutoModel.from_pretrained("bert-base-uncased")

# Initialize the classification model
model = BERTForClassification(bert_model)

# Define the sample input and ensure it's suitable for your model's expected input
sample_input = {'input_ids': tf.random.uniform(shape=(1, 128), minval=0, maxval=30522, dtype=tf.int32)}  # Adjusted maxval to match BERT's vocab size

# Build the model by calling it on a sample input
model(sample_input)

# Now you can print the summary
model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "bert_for_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_2 (Dropout)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 1538 (6.01 KB)
Non-trainable params: 109482240 (417.64 MB)
_________________________________________________________________


In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, epochs=3, validation_data=test_dataset)


Epoch 1/3
 7194/40000 [====>.........................] - ETA: 6:48:57 - loss: 0.7247 - accuracy: 0.5210

In [ ]:
# Evaluate the model on the test set
results = model.evaluate(test_dataset)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")


In [ ]:
from sklearn.metrics import f1_score
import numpy as np

# Assuming you have a test_dataset without batching or you can adjust the batch size accordingly
y_true = np.concatenate([y for x, y in test_dataset], axis=0)
y_pred = model.predict(test_dataset)
y_pred = np.argmax(y_pred, axis=1)

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='macro')  # You can change the averaging method to 'micro', 'macro', 'weighted', or others
print('F1 Score:', f1)


 875/1250 [====================>.........] - ETA: 1:25

KeyboardInterrupt: 